In [60]:
%load_ext autoreload
%autoreload 2

import inspect
from typing import NamedTuple, Union, Callable
from src.dataset_builders.text_data import create_easy_token_extraction_data
from src.nn_utils.rnn_encoder import Encoder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Simple example

In [154]:
def param_maker(foo, bar):
    print("running param_maker")
    param = {str(foo): 0,
             str(bar): 1,
            }
    return param
    
def func(foo=5, bar="nope", **kwargs):
    print( 10+foo, bar*2)
    # print(param_from_function.keys())

class Config(NamedTuple):
    print("preparing Config")
    bar: str = "BAR"
    foo: int = 25
    param_from_function: dict = param_maker(foo, bar)
    print("Config Done\n")
    

class Config_1(NamedTuple):
    print("preparing Config_1")
    bar: str = "BAR"
    foo: int = 25
    param_func: Callable = param_maker
    param_from_function: dict = None #param_func(foo, bar)
    print("Config Done\n")


## Ugly, but at least it works
class Config_2(Config_1):
    def activate(self):
        conf_dict = self._asdict()
        conf_dict["param_from_function"] = self.param_func(self.foo, self.bar)
        conf = Config_1(**conf_dict)
        return conf

        
class Config_3:
    ## pass in **cnf (where cnf is an instance of Config_1)
    def __init__(self, **kwargs):
        print("Initializing Config_3")
        cnf = Config_1(**kwargs)
        cnf_dict = cnf._asdict()
        
        print("Activating Config_3")
        cnf_dict["param_from_function"] = cnf.param_func(cnf.foo, cnf.bar)
        
        for item in cnf_dict.items():
            setattr(self, *item)
        print()
        
        



preparing Config
running param_maker
Config Done

preparing Config_1
Config Done



In [166]:
## Yet Another way to instantiate a Config Object

class Config_4:
    print("preparing Config_4")
    bar = "BAR"
    foo = 25
    # param_func = param_maker
    param_from_function = None #param_func(foo, bar)

    def __init__(self, **kwargs):
        print("Initializing/Acitvating Config_4")
        for item in kwargs.items():
            setattr(self, *item)
        foo = self.foo
        bar = self.bar
        param_func = self.param_func
        self.param_from_function = param_func(foo, bar)
        print("Config_4 Done\n")
    
    def param_func(self, foo, bar):
        param = param_maker(foo, bar)
        return param
    
cnf = Config_4(foo=100, bar="yep")

cnf.param_from_function

preparing Config_4
Initializing/Acitvating Config_4
running param_maker
Config_4 Done



{'100': 0, 'yep': 1}

# Question:
```
Any way to get NamedTuple (or other config object) to run functions on instansiation? / only on instantiation?
```

In [146]:
# Both statements below should produce
# {'100': 0, 'yep': 1}


# Config Does not run funcs with passed values
cnf = Config(foo=100, bar="yep")
print("param_from_function in instance of Config")
print(cnf.param_from_function, "\n")

print()
print(param_maker(cnf.foo, cnf.bar))


## Config_2
## Two steps and the neccessity of writing a somewhat clunky activation function
## But a single class instantiation
# print("-"*30, "\n")
# print("Config_2")
# cnf = Config_2(foo=100, bar="yep")
# print("in cnf UNactivated instance of Config_2")
# print(cnf.param_from_function, "\n")

# print("in cnf activated instance of Config_2")
# cnf = cnf.activate()
# print(cnf.param_from_function, "\n")


## Config_3
## One instantiation, no need for separate activation
## requires writing a funny looking __init__
## loose NamedTuple immutability
# print("-"*30, "\n")
# cnf = Config_3(foo=100, bar="yep")
# print("param_from_function in instance of Config_3")
# print(cnf.param_from_function, "\n")


## Config_4
## 
print("-"*30, "\n")
cnf = Config_4(foo=100, bar="yep")
print("param_from_function in instance of Config_4")
print(cnf.param_from_function, "\n")

param_from_function in instance of Config
{'25': 0, 'BAR': 1} 


running param_maker
{'100': 0, 'yep': 1}
------------------------------ 

Initializing/Acitvating Config_4
val yep
arg bar
kwargs {'foo': 100, 'bar': 'yep'}
Config_4 Done

param_from_function in instance of Config_4
None 



# More Realish Example

In [ ]:
def build_vocab(voc_input, build_from="string"):
    # characters: list = ["[STOP]", "[START]", "[UNK]"]
    # characters: list = ["[PAD]", "[STOP]", "[START]", "[UNK]"]
    characters: list = ["[STOP]", "[UNK]"]
    if build_from == "string":
        characters += [ch for ch in voc_input]
        vocabulary: dict = {}
        for ii, ch in enumerate(characters):
            vocabulary[ch] = ii
    else:
        print("build_from must be one of ['string']")
    return vocabulary

class DataConfig(NamedTuple):
    data_size: int = 50000
    char_string: str = "abcdeXY"
    prefix_len_range: tuple = (2,3)
    suffix_len_range: tuple = (2,3)
    token_len_range: tuple = (2,3)
    
# class EncoderConfig(NamedTuple):
#     pass

class NNConfig(NamedTuple):
   
    batch_size = 32  # Batch size for training.
    epochs = 900  # Number of epochs to train for.
    latent_dim = 16  # Latent dimensionality of the encoding space.
    
    print("Creating Data set")
    data_config = DataConfig()
    data = create_easy_token_extraction_data(**data_config._asdict())

    print("Building Vocabulary")
    vocabulary_map: dict = build_vocab(data_config.char_string)
    num_input_characters: int = len(vocabulary_map)
    num_output_characters: int = num_input_characters
    
    print("Building Encoding")
    encoder = Encoder(vocabulary_map)
    input_encoding = encoder.encode(data[:, 0], encode_as="input")
    output_encoding = encoder.encode(data[:, 1], encode_as="output")
